# Mixed NB (Gaussian+Multinomial)
* gnb : 'avg_rating' 피쳐 한개만
* mnb : alpha = 0.15(cv=5, 범위 0-1까지)
    * 0.32(cv=3, 범위 0-1까지)
    * 1.87(cv=3, 범위 0-10까지)

In [1]:
df = pd.read_csv('../resource/final_df1.csv')
y = df['rating(y)'].values
real_X = df[['avg_rating']].values
cat_X = df.ix[:,'0':].values

In [2]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
gnb = GaussianNB()
mnb = MultinomialNB(alpha = 0.15)
#mnb = MultinomialNB(alpha = 0.32)
#mnb = MultinomialNB(alpha = 1.87)

### Gaussian, Multinomial 모델 fitting

In [3]:
gnb.fit(real_X, y)
gnb_pred = gnb.predict(real_X)
gnb_prob = gnb.predict_proba(real_X)

mnb.fit(cat_X, y)
mnb_pred = mnb.predict(cat_X)
mnb_prob = mnb.predict_proba(cat_X)

### Mixed 모델의 조건부확률

In [4]:
mix_prob = np.multiply(gnb_prob, mnb_prob)
mix_prob.shape

(544L, 5L)

### Mixed 모델의 조건부확률 조정(softmax)

In [5]:
def softmax(w, t=1.0):
    """Calculate the softmax of a list of numbers w.

    Parameters
    ----------
    w : list of numbers
    t : float

    Return
    ------
    a list of the same length as w of non-negative numbers

    Examples
    --------
    >>> softmax([0.1, 0.2])
    array([ 0.47502081,  0.52497919])
    >>> softmax([-0.1, 0.2])
    array([ 0.42555748,  0.57444252])
    >>> softmax([0.9, -10])
    array([  9.99981542e-01,   1.84578933e-05])
    >>> softmax([0, 10])
    array([  4.53978687e-05,   9.99954602e-01])
    """
    e = np.exp(np.array(w) / t)
    dist = e / np.sum(e)
    return dist

In [6]:
for i in range(544):
    print(softmax(mix_prob[i]))

[ 0.18908658  0.18921673  0.21981379  0.20976562  0.19211729]
[ 0.19016094  0.24788545  0.18920208  0.18651647  0.18623506]
[ 0.1816712   0.18188007  0.18295687  0.2715965   0.18189536]
[ 0.19749695  0.2356166   0.18948584  0.18870766  0.18869295]
[ 0.1811507   0.18231431  0.27207366  0.18356907  0.18089226]
[ 0.20137119  0.21121786  0.1959288   0.19574128  0.19574087]
[ 0.19187951  0.21340272  0.20795711  0.19490129  0.19185937]
[ 0.18730041  0.18757414  0.2396283   0.1976722   0.18782495]
[ 0.19266275  0.20953057  0.20776665  0.19724948  0.19279055]
[ 0.18813793  0.1914513   0.19662699  0.23558286  0.18820091]
[ 0.18387494  0.18422638  0.18811288  0.25864521  0.18514059]
[ 0.18073175  0.18186514  0.27521147  0.18195601  0.18023563]
[ 0.18938957  0.18968037  0.23494206  0.19496457  0.19102343]
[ 0.18478206  0.18561857  0.24534898  0.19891501  0.18533538]
[ 0.1848754   0.18691655  0.26151499  0.18379327  0.18289979]
[ 0.18192339  0.1826422   0.26970414  0.18381842  0.18191185]
[ 0.1841

In [7]:
mix_prob_softmax = np.zeros((544, 5))
for i in range(544):
    mix_prob_softmax[i] = softmax(mix_prob[i])
mix_prob_softmax

array([[ 0.18908658,  0.18921673,  0.21981379,  0.20976562,  0.19211729],
       [ 0.19016094,  0.24788545,  0.18920208,  0.18651647,  0.18623506],
       [ 0.1816712 ,  0.18188007,  0.18295687,  0.2715965 ,  0.18189536],
       ..., 
       [ 0.18194145,  0.1820215 ,  0.18392407,  0.26932338,  0.1827896 ],
       [ 0.18973559,  0.1904562 ,  0.24554976,  0.1873022 ,  0.18695625],
       [ 0.18796056,  0.1884653 ,  0.20171865,  0.23420357,  0.18765192]])

In [8]:
np.sum(mix_prob_softmax[0])

1.0

### Mixed 모델의 최종 predict

In [9]:
mix_pred = np.zeros(544, )
for i in range(544):
    mix_pred[i] = np.argmax(mix_prob_softmax[i])
mix_pred += 1 # 별점은 1점부터 5점까지 줄수있음(int)
mix_pred

array([ 3.,  2.,  4.,  2.,  3.,  2.,  2.,  3.,  2.,  4.,  4.,  3.,  3.,
        3.,  3.,  3.,  4.,  4.,  3.,  3.,  3.,  4.,  3.,  2.,  1.,  3.,
        3.,  3.,  4.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,  3.,  3.,
        4.,  4.,  5.,  3.,  4.,  5.,  4.,  3.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  2.,  4.,  3.,  2.,  4.,  3.,  2.,  1.,  1.,
        2.,  2.,  4.,  1.,  2.,  3.,  5.,  3.,  3.,  2.,  4.,  3.,  2.,
        3.,  5.,  4.,  3.,  3.,  2.,  3.,  1.,  3.,  3.,  2.,  3.,  1.,
        1.,  2.,  2.,  1.,  2.,  1.,  2.,  2.,  3.,  4.,  3.,  4.,  3.,
        4.,  3.,  4.,  3.,  4.,  2.,  4.,  3.,  4.,  3.,  1.,  2.,  2.,
        2.,  3.,  3.,  3.,  3.,  3.,  4.,  2.,  4.,  3.,  3.,  4.,  3.,
        4.,  3.,  3.,  2.,  2.,  4.,  4.,  3.,  1.,  2.,  2.,  3.,  2.,
        4.,  2.,  5.,  4.,  5.,  3.,  4.,  3.,  5.,  3.,  3.,  3.,  3.,
        3.,  3.,  4.,  2.,  3.,  3.,  4.,  3.,  2.,  2.,  3.,  5.,  5.,
        4.,  2.,  3.,  4.,  3.,  1.,  3.,  3.,  3.,  2.,  3.,  3

### score 계산

In [10]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(mix_pred, y)

0.095588235294117641

In [11]:
print(np.max(mix_pred), np.mean(mix_pred), np.min(mix_pred), np.std(mix_pred))

5.0 3.09375 1.0 1.06258109552
